# Clustering

<img align="right" style="padding-left:10px; height: 20%; width: 20%;" src="figures/raindrops.jpg">

* Clustering is the process of discovering categories of data in a multi-dimensional dataset. 
* There are several kinds of clustering possible, we study just one type. 
* *Agglomerative clustering* builds categories of data points based upon their distance from each other according to some metric.  It groups the two closest points, and continues doing this until the result is one big blob. One can then divide the data into any number of categories by dividing up the agglomeration. _Sort of like, after some rain, how raindrops coalesce!_
* Clustering is an example of *unsupervised learning*. There is no preconceived notion of the categories that should arise. Contrast with classification, where there _is_ a preconceived notion of the categories.

This code is adapted from: https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/

# First, let's input some 2-d data. 

In [ ]:
import numpy as np

data = np.array([[5, 3],
                 [10, 15],
                 [70, 55],
                 [15, 12],
                 [71, 80],
                 [24, 10],
                 [30, 30],
                 [85, 70],
                 [60, 78],
                 [80, 91], ])
data

# Then let's label rows for a demo

* This is a trick to label points with characters
* `ord(character)` is the integer offset of `character` in ASCII. 
* `chr(integer)` is the character corresponding to the offset `integer`. 
* `chr(ord('a')+i)` is the `i`th character in the alphabet, starting at `i=0`. 

In [ ]:
row_labels = [chr(c+ord('a')) for c in range(len(data))]
row_labels

# Let's put this into a DataFrame 

In [ ]:
import pandas as pd
df = pd.DataFrame(data, index=row_labels, columns=('x', 'y'))
df

# Caveat about pandas and numpy
* It is good to remember that `numpy` predates `pandas`. 
* Some libraries take only `numpy` input. 
* Others tolerate either `pandas` or `numpy` input. 
* In this case, we have libraries that tolerate both formats. 

# Let's visualize the data 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))  # set size of overall figure
plt.subplots_adjust(bottom=0.1)  

# make a scatterplot of all data,
plt.scatter(df.loc[:,'x'],df.loc[:,'y'], label='True Position')

# label points with their letters for clarity. 
# use parallel iteration over three parallel lists 
for label, x, y in zip(df.index, df.loc[:, 'x'], df.loc[:, 'y']):  
    plt.annotate(
        label, # label of text
        xy=(x, y),  # where point is in coordinate space 
        xytext=(-3, 3),  # offset of label in pixels
        textcoords='offset points', # how to interpret coordinates 
        ha='right',  # text horizontal alignment
        va='bottom')  # text vertical alignment

# Some basic observations
* It's pretty obvious that this data is segregated into two categories. 
* It's not so obvious as to how to do this by machine. 
* The following is an illustration of one way to do it. 

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt

# make a hierarchical structure from the data. 
linked = linkage(data, 'single')

# plot the hierarchy as a 'dendrogram' 
plt.figure(figsize=(9, 6))  
_ = dendrogram(linked,  
               orientation='top',
               labels=row_labels,
               distance_sort='descending',
               show_leaf_counts=True)

# Agglomerative clustering
The above diagram is made as follows: 
1. Start with nothing linked. Every point is in its own cluster. 
2. Link the two closest clusters to one another. The distance between clusters is the minimum distance between points in the cluster. 
3. Continue until you have one large cluster.  

We usually skip showing the previous diagram, and proceed as follows: 

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Compute clusters
cluster2 = AgglomerativeClustering(n_clusters=2, affinity='euclidean')
cluster2.fit_predict(df)

# color each cluster with a different color of the rainbow
plt.figure(figsize=(6, 6))
plt.scatter(df.loc[:, 'x'],  # x coordinate
            df.loc[:, 'y'],  # y coordinate
            c=cluster2.labels_,  # how to determine colors of points.
            cmap='rainbow')  # what color code to use

# Label data points with their letters
for label, x, y in zip(df.index, df.loc[:, 'x'], df.loc[:, 'y']):
    plt.annotate(
        label,  # what label to place
        xy=(x, y),  # where the point is (actual coordinates)
        xytext=(-3, 3),  # where the characters should start (pixels)
        textcoords='offset points',  # use the xytext offsets to place text
        ha='right',  # horizontal alignment of text
        va='bottom')  # vertical alignment of text

# How this was done
* in `plt.scatter`, change the color for each cluster: `c=cluster2.labels_`
* Map each cluster to a color of the rainbow: `cmap='rainbow'`
        
Here's the same chart for _three clusters_.

In [ ]:
cluster3 = AgglomerativeClustering(n_clusters=3, affinity='euclidean')  
cluster3.fit_predict(df)  

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(df.loc[:,'x'],df.loc[:,'y'], c=cluster3.labels_, cmap='rainbow')  
for label, x, y in zip(df.index, df.loc[:, 'x'], df.loc[:, 'y']):  
    plt.annotate(
        label,
        xy=(x, y), 
        xytext=(-3, 3),
        textcoords='offset points', 
        ha='right', 
        va='bottom')

# Data is often not so well behaved
* There can be nore than two axes. 
* Clusters may not be separable. 
* Human-specified and data-driven categories need not match. 
* The reason is the scourge of the data scientist: *hidden variables*.

# Hidden variables
* Often, the human categorization of something differs from the machine categorization due to specialized knowledge. 
* No dataset is complete. 
* There are aspects that are not recorded. 
* These can determine categories that the data doesn't depict. 

# Let's try this out
* Here is a practical case study of hidden variables. 
* The Iris dataset doesn't in any sense contain everything about irises. 
* We can try clustering it, and compare that with the human categorization. 
* We'll get somewhat different results. 

In [ ]:
# Don't change this cell; just run it. 
from client.api.notebook import Notebook
ok = Notebook('Clustering.ok')

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
print(iris.DESCR)

1. Call Agglomerative Clustering on the iris data, as above, with three categories. Put the result into the variable `cluster`. 

In [ ]:
# Your answer: 
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean')  
cluster.fit_predict(iris.data)
cluster.labels_

In [ ]:
_ = ok.grade('q01')

Let's compare this with the expert decisions in `iris.target` 
Category numbers are arbitrary. You'll need to map between numbering systems to compare these. 

2. Compare the above output with `iris.target` to determine the closest match between category numbers. 
What category numbers in your map correspond to the ones in `iris.target`? 

___Your answer:___
* `iris.target` category 0 corresponds to `cluster.labels_` category ______1__________.
* `iris.target` category 1 corresponds to `cluster.labels_` category ______0___________.
* `iris.target` category 2 corresponds to `cluster.labels_` category ______2___________.

This situation is very common. The two categorizations are slightly different. We want to know how close they can be matched to one another. This is a measure of how much information is hidden that we are not recording or using in clustering. 

3. Create an `array` `comparable` that maps categories you have listed above to their closest equivalents in `iris.target`. You can do this with a `for` loop or become more clever. 

In [ ]:
# Your answer: 
import numpy as np
out = list()
for i in cluster.labels_: 
    if i == 1: 
        out.append(0)
    elif i==0: 
        out.append(1)
    else: 
        out.append(2)
comparable = np.array(out)
comparable

In [ ]:
_ = ok.grade('q03')

4. Compute the number of identical characterizations. Hint: use array comparison, and note that the sum of a Boolean vector is the number of True values. Put the result in `same`

In [ ]:
# Your answer: 
same = (comparable == iris.target).sum()
same

It's important to understand that this isn't an error of clustering. Clustering acts on the data it gets. The problem is that something is missing. What is missing distinguishes two categories of `iris.target` more clearly. 

4. Which categories in `iris.target` are not clearly distinguished? 

___Your answer:___

Well, we know something that's missing: the human categorization! Will adding that to the data fix the clustering? Let's try it as an educational exercise. 

5. Run the same experiment on the array created by combining `iris.data` and `iris.target` so that we have five columns of input data. Print the number of differences 

In [ ]:
# Your answer: 
newdata = np.insert(iris.data,4,iris.target, axis=1)
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean')  
cluster.fit_predict(newdata)
out = list()
for i in cluster.labels_: 
    if i == 1: 
        out.append(0)
    elif i==2: 
        out.append(1)
    else: 
        out.append(2)
cats = np.array(out)
same = (cats == iris.target).sum()
same

The moral of this story is that no clustering mechanism is perfect, and no clustering mechanism truly substitutes for human categorization. Clustering mechanisms can get close, but are never completely accurate. The last experiment begs the question. 